# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc168398940>
├── 'a' --> tensor([[ 0.5924, -0.5358, -0.4666],
│                   [-1.1082,  0.4625,  0.0257]])
└── 'x' --> <FastTreeValue 0x7fc0c507ae80>
    └── 'c' --> tensor([[ 0.2256,  0.3302, -0.0040,  1.1393],
                        [-1.8591, -1.5914, -1.5168,  2.5324],
                        [-1.4130, -0.3377,  0.2326,  0.0091]])

In [4]:
t.a

tensor([[ 0.5924, -0.5358, -0.4666],
        [-1.1082,  0.4625,  0.0257]])

In [5]:
%timeit t.a

63.7 ns ± 0.0591 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc168398940>
├── 'a' --> tensor([[-0.6935, -0.6065, -0.5212],
│                   [ 0.3373,  0.6014, -0.2533]])
└── 'x' --> <FastTreeValue 0x7fc0c507ae80>
    └── 'c' --> tensor([[ 0.2256,  0.3302, -0.0040,  1.1393],
                        [-1.8591, -1.5914, -1.5168,  2.5324],
                        [-1.4130, -0.3377,  0.2326,  0.0091]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 0.0787 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5924, -0.5358, -0.4666],
               [-1.1082,  0.4625,  0.0257]]),
    x: Batch(
           c: tensor([[ 0.2256,  0.3302, -0.0040,  1.1393],
                      [-1.8591, -1.5914, -1.5168,  2.5324],
                      [-1.4130, -0.3377,  0.2326,  0.0091]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5924, -0.5358, -0.4666],
        [-1.1082,  0.4625,  0.0257]])

In [11]:
%timeit b.a

65.2 ns ± 0.044 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1281,  1.0167,  0.3518],
               [-0.0946,  0.7074, -1.0512]]),
    x: Batch(
           c: tensor([[ 0.2256,  0.3302, -0.0040,  1.1393],
                      [-1.8591, -1.5914, -1.5168,  2.5324],
                      [-1.4130, -0.3377,  0.2326,  0.0091]]),
       ),
)

In [13]:
%timeit b.a = new_value

497 ns ± 0.186 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

813 ns ± 1.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 18.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 252 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 791 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc0b36c2b50>
├── 'a' --> tensor([[[ 0.5924, -0.5358, -0.4666],
│                    [-1.1082,  0.4625,  0.0257]],
│           
│                   [[ 0.5924, -0.5358, -0.4666],
│                    [-1.1082,  0.4625,  0.0257]],
│           
│                   [[ 0.5924, -0.5358, -0.4666],
│                    [-1.1082,  0.4625,  0.0257]],
│           
│                   [[ 0.5924, -0.5358, -0.4666],
│                    [-1.1082,  0.4625,  0.0257]],
│           
│                   [[ 0.5924, -0.5358, -0.4666],
│                    [-1.1082,  0.4625,  0.0257]],
│           
│                   [[ 0.5924, -0.5358, -0.4666],
│                    [-1.1082,  0.4625,  0.0257]],
│           
│                   [[ 0.5924, -0.5358, -0.4666],
│                    [-1.1082,  0.4625,  0.0257]],
│           
│                   [[ 0.5924, -0.5358, -0.4666],
│                    [-1.1082,  0.4625,  0.0257]]])
└── 'x' --> <FastTreeValue 0x7fc0b36c2be0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 44.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc0b36c2a60>
├── 'a' --> tensor([[ 0.5924, -0.5358, -0.4666],
│                   [-1.1082,  0.4625,  0.0257],
│                   [ 0.5924, -0.5358, -0.4666],
│                   [-1.1082,  0.4625,  0.0257],
│                   [ 0.5924, -0.5358, -0.4666],
│                   [-1.1082,  0.4625,  0.0257],
│                   [ 0.5924, -0.5358, -0.4666],
│                   [-1.1082,  0.4625,  0.0257],
│                   [ 0.5924, -0.5358, -0.4666],
│                   [-1.1082,  0.4625,  0.0257],
│                   [ 0.5924, -0.5358, -0.4666],
│                   [-1.1082,  0.4625,  0.0257],
│                   [ 0.5924, -0.5358, -0.4666],
│                   [-1.1082,  0.4625,  0.0257],
│                   [ 0.5924, -0.5358, -0.4666],
│                   [-1.1082,  0.4625,  0.0257]])
└── 'x' --> <FastTreeValue 0x7fc0b36c2c70>
    └── 'c' --> tensor([[ 0.2256,  0.3302, -0.0040,  1.1393],
                        [-1.8591, -1.5914, -1.5168,  2.5324],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 61.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.8 µs ± 80.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2256,  0.3302, -0.0040,  1.1393],
                       [-1.8591, -1.5914, -1.5168,  2.5324],
                       [-1.4130, -0.3377,  0.2326,  0.0091]],
              
                      [[ 0.2256,  0.3302, -0.0040,  1.1393],
                       [-1.8591, -1.5914, -1.5168,  2.5324],
                       [-1.4130, -0.3377,  0.2326,  0.0091]],
              
                      [[ 0.2256,  0.3302, -0.0040,  1.1393],
                       [-1.8591, -1.5914, -1.5168,  2.5324],
                       [-1.4130, -0.3377,  0.2326,  0.0091]],
              
                      [[ 0.2256,  0.3302, -0.0040,  1.1393],
                       [-1.8591, -1.5914, -1.5168,  2.5324],
                       [-1.4130, -0.3377,  0.2326,  0.0091]],
              
                      [[ 0.2256,  0.3302, -0.0040,  1.1393],
                       [-1.8591, -1.5914, -1.5168,  2.5324],
                       [-1.4130, -0.3377,  0.2326,  0.0091]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2256,  0.3302, -0.0040,  1.1393],
                      [-1.8591, -1.5914, -1.5168,  2.5324],
                      [-1.4130, -0.3377,  0.2326,  0.0091],
                      [ 0.2256,  0.3302, -0.0040,  1.1393],
                      [-1.8591, -1.5914, -1.5168,  2.5324],
                      [-1.4130, -0.3377,  0.2326,  0.0091],
                      [ 0.2256,  0.3302, -0.0040,  1.1393],
                      [-1.8591, -1.5914, -1.5168,  2.5324],
                      [-1.4130, -0.3377,  0.2326,  0.0091],
                      [ 0.2256,  0.3302, -0.0040,  1.1393],
                      [-1.8591, -1.5914, -1.5168,  2.5324],
                      [-1.4130, -0.3377,  0.2326,  0.0091],
                      [ 0.2256,  0.3302, -0.0040,  1.1393],
                      [-1.8591, -1.5914, -1.5168,  2.5324],
                      [-1.4130, -0.3377,  0.2326,  0.0091],
                      [ 0.2256,  0.3302, -0.0040,  1.1393],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

330 µs ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
